In [1]:
# Add directory above current directory to path
import sys as SYS; SYS.path.insert(0, '..')

# for saving
import os

from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt

from src import setup
# setup.use_gpu()

In [2]:
from src import ( 
    measurements as measure,
    density_matrix as DM,
    simulation as sim,
    orders,
    order_rules,
    random_unitary,
    simulation)

from Scripts import simulation_CLI as cleo

In [3]:
all_orders = orders.all_c5_orders(8, 2)
prev_pops = [0.11999999, 0.22      , 0.32      , 0.42000002, 0.4       ,
       0.25, 0.23      , 0.21      ]
pops = [0.32371777, 0.22727564, 0.23996802, 0.29631424, 0.19628224,
       0.37368578, 0.22272438, 0.290032  ]
past_order = all_orders[0]

In [100]:
num_qubits = 8
def mimic(past_order, prev_pops, pops, connectivity, sub_unitary):
    """
    Args:
        two_qubit_dms_current: the current two qubit density matrices
        two_qubit_dms_previous: the previous two qubit density matrices
        pops: the current one qubit populations
        prev_pops: the previous one qubit populations
        past_order: the previous order
        connectivity: a string representing the connectivity of the qubits
        sub_unitary: the unitary that operates on a group
        dm: the current density matrix
    """

    num_qubits = len(pops)
    chunk_size = 2

    # this is inefficient, dont need to recalculate every time
    match connectivity:
        case 'c5':
            all_orders = orders.all_c5_orders(num_qbits=num_qubits, chunk_size=chunk_size)
        case 'c6':
            all_orders = orders.all_c6_orders(num_qbits=num_qubits, chunk_size=chunk_size)
        case 'c7':
            all_orders = orders.all_c7_orders(num_qbits=num_qubits, chunk_size=chunk_size)
        case _:
            raise ValueError(f"connectivity {connectivity} not recognized")

    all_qubits = set([i for i in range(num_qubits)])

    #find all neighbours of a qubit
    def find_shared_elements(list_of_lists_of_lists, target_element):
        shared_elements = set()  # Use a set to store unique elements
        for sublist in list_of_lists_of_lists:
            for subsublist in sublist:
                if target_element in subsublist:
                    shared_elements.update([elem for elem in subsublist if elem != target_element])
        return list(shared_elements)

    #list of lists where each list is the set of neighbours of the qubit with that list index. ie the nth list is a list of neighbours of the nth qubit on landscape
    neighbours_qubit_index = []
    for id in range(num_qubits):
        neighbours_qubit_index.append(find_shared_elements(all_orders,id))

    extractable_work_i0 = np.array(
        measure.extractable_work_of_each_qubit_from_pops(prev_pops))
    extractable_work_i1 = np.array(
        measure.extractable_work_of_each_qubit_from_pops(pops))
    change_in_ex_work_prev_step = extractable_work_i1-extractable_work_i0

    #code to find the qubit the target qubit was paired in the previous order
    def paired_element(given_element):
        for pair in past_order:
            # Convert the NumPy array to a list
            pair_list = pair.tolist()
            if given_element in pair_list:
                # Return the other element in the pair
                return pair_list[1 - pair_list.index(given_element)]
        # If the given element is not found in any pair, return None
        return None

    #code to find the qubit indices with change in extractable work in ascending order. These will help find the decider qubits
    def find_indices_in_ascending_order(values):
        # Enumerate the values to get (index, value) pairs
        indexed_values = list(enumerate(values))

        # Sort the indexed values based on the values (the second element of each tuple)
        sorted_indices = sorted(indexed_values, key=lambda x: x[1])

        # Extract the indices from the sorted list of (index, value) pairs
        indices_in_ascending_order = [index for index, _ in sorted_indices]

        return indices_in_ascending_order

    decider_Q_index = find_indices_in_ascending_order(change_in_ex_work_prev_step)
    print(decider_Q_index)

    #code to find allowed neighbourhoods after some have been fixed 
    
    order = []
    def find_lists_with_sublist(list_of_lists_of_lists, sublist_to_match):
        matching_lists = []
        for lists_of_lists in list_of_lists_of_lists:
            for sublist in lists_of_lists:
                if np.array_equal(sublist, sublist_to_match):
                    matching_lists.append(lists_of_lists)
                    break  # Break out of the inner loop once a match is found
        return matching_lists
        
    for qubit_id in decider_Q_index:
        if qubit_id not in order:
            print(qubit_id)
            sub_order = []
            max_D_W_ex = float('-inf')
            if len(order) > 0: 
                for s in order:
                    print(s)
                    s = np.array(s)
                    match = find_lists_with_sublist(all_orders, s)
                    allowed_orders = match
                    if allowed_orders:  # Check if any matching sublist is found
                        return allowed_orders 
            else: 
                allowed_orders = all_orders
            for id in range(num_qubits):
                    neighbours_qubit_index.append(find_shared_elements(allowed_orders,id))
            neighbours = neighbours_qubit_index[qubit_id]
            neighbours = [x for x in neighbours if x not in np.array(order).flatten()]
            print(neighbours)
            for neighbour in neighbours:
                if max_D_W_ex <= change_in_ex_work_prev_step[neighbour]:
                    max_D_W_ex=change_in_ex_work_prev_step[neighbour]
                    neighbour_to_mimic = neighbour
                    pop_diff_to_mimic = prev_pops[neighbour_to_mimic] - prev_pops[paired_element(neighbour_to_mimic)]
            diff = 0
            for neighbour in neighbours:
                pop_diff = pops[qubit_id] - pops[neighbour]
                diff_with_n=pop_diff - pop_diff_to_mimic
                #print(diff_with_n)
                if diff_with_n  <= diff:
                    diff= diff_with_n
                    Q_pair = neighbour
                    sub_order = [qubit_id,Q_pair]
                    order.append(sub_order)
                    print(order)

    current_order = order
    return current_order

In [101]:
mimic(past_order, prev_pops, pops, "c5", "haar2Qunitary")

[0, 5, 7, 1, 6, 2, 3, 4]
0
[1, 3]
-0.0035578899999999525
[[0, 1]]
-0.07259648999999996
[[0, 1], [0, 3]]
5
[0, 1]


[array([[0, 1],
        [4, 5],
        [2, 3],
        [6, 7]])]

In [32]:
all_orders

[array([[0, 1],
        [4, 5],
        [2, 3],
        [6, 7]]),
 array([[3, 0],
        [7, 4],
        [1, 2],
        [5, 6]])]

In [85]:
all_orders = orders.all_c5_orders(8, 2)
matchs=[]
suborder = [[0,1],[4,5]]

for element in suborder:
    for order in all_orders:
        if element in order:
            matchs.append(order)
            print(order)
            matchs
    #print(matchs)
    


[[0 1]
 [4 5]
 [2 3]
 [6 7]]
[[0 1]
 [4 5]
 [2 3]
 [6 7]]


In [48]:
find_lists_with_sublist(all_orders,[[0,1],[4,5]])

TypeError: 'numpy.int64' object is not iterable

In [78]:
def find_lists_with_sublist(list_of_lists_of_lists, sublist_to_match):
    matching_lists = []
    print("Sublist to match:", sublist_to_match)
    print("Type of sublist to match:", type(sublist_to_match))
    for lists_of_lists in list_of_lists_of_lists:
        for inner_sublist in sublist_to_match:
            print("Inner sublist:", inner_sublist)
            inner_sublist_set = {frozenset(map(tuple, inner_sublist))}
            for sublist in lists_of_lists:
                print("Sublist:", sublist)
                sublist_set = {frozenset(map(tuple, pair)) for pair in sublist}
                if inner_sublist_set.issubset(sublist_set):
                    matching_lists.append(sublist)
                    break  # No need to continue searching in this list of lists
    return matching_lists

all_orders = [
    np.array([[0, 1], [4, 5], [2, 3], [6, 7]]),
    np.array([[3, 0], [7, 4], [1, 2], [5, 6]])
]
sublist_to_match = [[0, 1], [4, 5]]
matching_lists = find_lists_with_sublist(all_orders, sublist_to_match)
print(matching_lists)

Sublist to match: [[0, 1], [4, 5]]
Type of sublist to match: <class 'list'>
Inner sublist: [0, 1]


TypeError: 'int' object is not iterable

In [71]:
def mimic(past_order, prev_pops, pops, connectivity, sub_unitary):
    num_qubits = len(pops)
    chunk_size = 2

    # Define all_orders based on connectivity
    match connectivity:
        case 'c5':
            all_orders = orders.all_c5_orders(num_qbits=num_qubits, chunk_size=chunk_size)
        case 'c6':
            all_orders = orders.all_c6_orders(num_qbits=num_qubits, chunk_size=chunk_size)
        case 'c7':
            all_orders = orders.all_c7_orders(num_qbits=num_qubits, chunk_size=chunk_size)
        case _:
            raise ValueError(f"connectivity {connectivity} not recognized")
     # this is inefficient, dont need to recalculate every time
    match connectivity:
        case 'c5':
            all_orders = orders.all_c5_orders(num_qbits=num_qubits, chunk_size=chunk_size)
        case 'c6':
            all_orders = orders.all_c6_orders(num_qbits=num_qubits, chunk_size=chunk_size)
        case 'c7':
            all_orders = orders.all_c7_orders(num_qbits=num_qubits, chunk_size=chunk_size)
        case _:
            raise ValueError(f"connectivity {connectivity} not recognized")

    all_qubits = set([i for i in range(num_qubits)])

    #find all neighbours of a qubit
    def find_shared_elements(list_of_lists_of_lists, target_element):
        shared_elements = set()  # Use a set to store unique elements
        for sublist in list_of_lists_of_lists:
            for subsublist in sublist:
                if target_element in subsublist:
                    shared_elements.update([elem for elem in subsublist if elem != target_element])
        return list(shared_elements)

    #list of lists where each list is the set of neighbours of the qubit with that list index. ie the nth list is a list of neighbours of the nth qubit on landscape
    neighbours_qubit_index = []
    for id in range(num_qubits):
        neighbours_qubit_index.append(find_shared_elements(all_orders,id))

    # Compute change in ex work for each qubit
    extractable_work_i0 = np.array(measure.extractable_work_of_each_qubit_from_pops(prev_pops))
    extractable_work_i1 = np.array(measure.extractable_work_of_each_qubit_from_pops(pops))
    change_in_ex_work_prev_step = extractable_work_i1 - extractable_work_i0

    # Function to find the paired element for a given element in past_order
    def paired_element(given_element):
        for pair in past_order:
            pair_list = pair.tolist()
            if given_element in pair_list:
                return pair_list[1 - pair_list.index(given_element)]
        return None

    # Function to find indices in ascending order of values
    def find_indices_in_ascending_order(values):
        indexed_values = list(enumerate(values))
        sorted_indices = sorted(indexed_values, key=lambda x: x[1])
        return [index for index, _ in sorted_indices]

    # Find qubit indices with change in ex work in ascending order
    decider_Q_index = find_indices_in_ascending_order(change_in_ex_work_prev_step)

    order = []
    for qubit_id in decider_Q_index:
        if qubit_id not in order:
            sub_order = []
            max_D_W_ex = float('-inf')
            if len(order) > 0:
                # Logic here
                pass
            else:
                allowed_orders = all_orders
            neighbours_qubit_index = []
            for id in range(num_qubits):
                neighbours_qubit_index.append(find_shared_elements(allowed_orders, id))
            neighbours = neighbours_qubit_index[qubit_id]
            neighbours = [x for x in neighbours if x not in np.array(order).flatten()]
            for neighbour in neighbours:
                if change_in_ex_work_prev_step[neighbour] > max_D_W_ex:
                    max_D_W_ex = change_in_ex_work_prev_step[neighbour]
                    neighbour_to_mimic = neighbour
            pop_diff_to_mimic = prev_pops[neighbour_to_mimic] - prev_pops[paired_element(neighbour_to_mimic)]
            diff = float('inf')
            for neighbour in neighbours:
                pop_diff = pops[qubit_id] - pops[neighbour]
                diff_with_n = pop_diff - pop_diff_to_mimic
                if diff_with_n <= diff:
                    diff = diff_with_n
                    Q_pair = neighbour
                    sub_order = [qubit_id, Q_pair]
            order.append(sub_order)
    return order